In [68]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [69]:
%load_ext autoreload
%autoreload 2

In [104]:
import numpy
import pandas as pd

import theano
import theano.tensor as T

In [172]:
data = pd.read_pickle('/home/hudson/experiments/prostate.df')
#print data.head(2)
#print data[data.columns[-1]]

train_set_y = (data.values[:, -1] == 'tumor')
train_set_x = data.values[:, :-1]
n_genes = train_set_x.shape[1]
print n_genes
print train_set_x.shape
print train_set_y.shape
train_set_x = train_set_x.astype(float)
#train_set_y = train_set_y.astype(float)
print train_set_y

batch_size = 10
n_batches = len(train_set_y)/batch_size
print len(train_set_y), n_batches

12533
(102, 12533)
(102,)
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
102 10


In [252]:
# Declare Theano symbolic variables
# all parameters are initialized to be zero; 
x = T.matrix("x")
y = T.vector("y")

# initialize the weight vector w randomly
#
# this and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)
#w = theano.shared(rng.randn(feats), name="w")
w = theano.shared(0., name="w")

# initialize the bias term
b = theano.shared(0., name="b")

print("Initial model:")
print(w.get_value())
print(b.get_value())


# Construct Theano expression graph
p_func = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability 

prediction = p_func > 0.5                    # The prediction thresholded
xent = -y * T.log(p_func) - (1-y) * T.log(1-p_func)  # Cross-entropy loss function
#cost = xent.mean() + 0.01 * (w ** 2).sum()     # The cost to minimize
cost = T.sum(-y * T.log(p_func) - (1-y) * T.log(1-p_func))


gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                          # (we shall return to this in a
                                          # following section of this tutorial)


#index = T.lscalar()                
# Compile
learning_rate = 0.1  # #1e-2
train = theano.function(
          inputs=[x,y],
          #utputs=cost,
          outputs=[prediction, xent], 
          updates=((w, w - learning_rate * gw), (b, b - learning_rate * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

# Train
training_steps = 5
for i in range(training_steps):     
    prediction, err = train_model(train_set_x, train_set_y)
    #print i, T.cast(cost, 'int32').eval()
    
print("Final model:")
print(w.get_value(), w.get_value().size)
print(b.get_value(), b.get_value().size)
print("target values for Data:")
print(train_set_y)
print("prediction on Data:")
print(predict(train_set_x))

Initial model:
0.0
0.0
Final model:
(0.0, 1)
(0.0, 1)
target values for Data:
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
prediction on Data:
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [253]:
w = theano.shared(rng.randn(n_genes), name="w")

# initialize the bias term
b = theano.shared(0., name="b")
print("Initial model:")
print(w.get_value())
print(b.get_value())


# Construct Theano expression graph
p_func = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability 

prediction = p_func > 0.5                    # The prediction thresholded
xent = -y * T.log(p_func) - (1-y) * T.log(1-p_func)  # Cross-entropy loss function
#cost = xent.mean() + 0.01 * (w ** 2).sum()     # The cost to minimize
cost = T.sum(-y * T.log(p_func) - (1-y) * T.log(1-p_func))


gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                          # (we shall return to this in a
                                          # following section of this tutorial)


#index = T.lscalar()                
# Compile
learning_rate = 0.1  # #1e-2
train = theano.function(
          inputs=[x,y],
          #utputs=cost,
          outputs=[prediction, xent], 
          updates=((w, w - learning_rate * gw), (b, b - learning_rate * gb)))
predict = theano.function(inputs=[x], outputs=prediction)


training_steps = 5
for i in range(training_steps):     
    prediction, err  = train_model(train_set_x, train_set_y)
    #print i, T.cast(cost, 'int32').eval()

    
    
print("Final model:")
print(w.get_value(), w.get_value().size)
print(b.get_value(), b.get_value().size)
print("target values for Data:")
print(train_set_y)
print("prediction on Data:")
print(predict(train_set_x))

Initial model:
[ 0.30332745  0.3210286  -0.76226472 ...,  1.2050331  -2.17524462
 -0.28026123]
0.0
Final model:
(array([ 0.30332745,  0.3210286 , -0.76226472, ...,  1.2050331 ,
       -2.17524462, -0.28026123]), 12533)
(0.0, 1)
target values for Data:
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
prediction on Data:
[0 0 0 0 0 1 1 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 0 1 0 1 1
 0 1 1 1 1 0 1 0 0 0 0 1 1 0 0 

In [255]:
#Add L1-regularization to your model, i.e., the optimization should minimize
# Declare Theano symbolic variables
x = T.matrix("x")
y = T.vector("y")

# initialize the weight vector w randomly
#
# this and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)
w = theano.shared(rng.randn(feats), name="w")
#w = theano.shared(0., name="w")

# initialize the bias term
b = theano.shared(0., name="b")

print("Initial model:")
print(w.get_value())
print(b.get_value())


# Construct Theano expression graph
p_func = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability 

prediction = p_func > 0.5                    # The prediction thresholded
xent = -y * T.log(p_func) - (1-y) * T.log(1-p_func)  # Cross-entropy loss function
#cost = xent.mean() + 0.01 * (w ** 2).sum()     # The cost to minimize
nnl = T.sum(-y * T.log(p_func) - (1-y) * T.log(1-p_func))
cost = nnl + c * T.sum(T.dot(x, w))


gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                          # (we shall return to this in a
                                          # following section of this tutorial)


#index = T.lscalar()                
# Compile
learning_rate = 0.1  # #1e-2
train = theano.function(
          inputs=[x,y],
          #utputs=cost,
          outputs=[prediction, xent], 
          updates=((w, w - learning_rate * gw), (b, b - learning_rate * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

# Train
training_steps = 5
for i in range(training_steps):     
    prediction, err = train_model(train_set_x, train_set_y)
    #print i, T.cast(cost, 'int32').eval()
    
print("Final model:")
print(w.get_value(), w.get_value().size)
print(b.get_value(), b.get_value().size)
print("target values for Data:")
print(train_set_y)
print("prediction on Data:")
print(predict(train_set_x))

Initial model:
[-0.35577105 -0.34849789  0.50024541 ...,  1.25354392 -0.5213866
  1.23277313]
0.0
Final model:
(array([-0.35577105, -0.34849789,  0.50024541, ...,  1.25354392,
       -0.5213866 ,  1.23277313]), 12533)
(0.0, 1)
target values for Data:
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
prediction on Data:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 1 1 0 1 1 1 0 1 0
 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1

In [51]:
class LogisticRegression(object):
    
    def __init__(self, input, n_in, n_out):
        self.input = input
        
        W_values = 4*numpy.random.uniform(
                low=-numpy.sqrt(6. / (n_in + n_out)),
                high=numpy.sqrt(6. / (n_in + n_out)),
                size=(n_in, n_out)
        )
        self.W = theano.shared(value=W_values, name='W', borrow=True)

        b_values = numpy.zeros((n_out,))
        self.b = theano.shared(value=b_values, name='b', borrow=True)
           